In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
feats_channels = 3
num_classess = 3
feats = torch.randn(2,3,128,128)
preds = torch.randn(2,3,128,128)

batch_size,num_channels,h,w = feats.size()
weight_cls = preds.permute(0,2,3,1)
weight_cls = weight_cls.reshape(-1,num_classess)
weight_cls = F.softmax(weight_cls,dim=-1)
# 主要是形成高斯分布的均值和方差，有几个class就有几组
memory = nn.Parameter(torch.cat([
    torch.zeros(num_classess,1,dtype=torch.float),torch.ones(num_classess,1,dtype=torch.float)],dim=1),requires_grad=False)
memory_means = memory.data[:,0]
memory_stds = memory.data[:,1]
memory_list = []

for idx in range(num_classess):
    torch.manual_seed(idx)
    cls_memory = torch.normal(mean=torch.full((1,feats_channels),memory_means[idx]),
                              std=torch.full((1,feats_channels),memory_stds[idx]))
    memory_list.append(cls_memory)

memory_tensor = torch.cat(memory_list,dim=0)
selected_memory = torch.matmul(weight_cls,memory_tensor)

selected_memory = selected_memory.view(batch_size,h,w,num_channels)
selected_memory = selected_memory.permute(0,3,1,2)
print(weight_cls.shape)
print(memory_tensor.shape)
print(selected_memory.shape)

torch.Size([32768, 3])
torch.Size([3, 3])
torch.Size([2, 3, 128, 128])


In [9]:
torch.full((1,256),0).shape


torch.Size([1, 256])